# Predicitng tasks from input data

## importing necessary modules

In [9]:
import os
ml_code_path='/home/leilapirhaji/mz_embed_engine/ml'
os.chdir(ml_code_path)

import pandas as pd
import importlib
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
import optuna
import imaplib


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import random
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score


#importing fundtion to get encoder info and perfrom tasks 
from finetune.get_finetune_encoder import  get_finetune_input_data


%who



DataLoader	 LabelEncoder	 TensorDataset	 X_data_test	 X_data_train	 X_data_val	 accuracy_score	 best_val_auc	 best_val_f1	 
f1_score	 finetune	 get_finetune_input_data	 imaplib	 importlib	 input_data_location	 label_encoder	 log_reg_multi_class	 ml_code_path	 
nn	 np	 optim	 optuna	 os	 pd	 precision_score	 random	 recall_score	 
results	 results_df	 roc_auc_score	 task	 task_list_cat	 task_list_survival	 test_auc	 test_f1	 torch	 
y_data_test	 y_data_train	 y_data_val	 


# Fine tune OG Splits

## importing input data 

In [10]:

#input data
#input_data_location='/home/leilapirhaji/PROCESSED_DATA_S_8.1.1'
input_data_location='/home/leilapirhaji/PROCESSED_DATA_finetune_OG_split'


#tasks to predict using encoder
task_list_cat=['Benefit BINARY', 'Nivo Benefit BINARY', 'MSKCC BINARY', 'IMDC BINARY', 'Benefit ORDINAL', 'MSKCC ORDINAL', 'IMDC ORDINAL', 'ORR', 'Benefit', 'IMDC', 'MSKCC', 'Prior_2' ]

#survival tasks
task_list_survival=[ 'OS', 'OS_Event', 'PFS', 'PFS_Event' ]


#get fine-tuning input data 
(X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=get_finetune_input_data(input_data_location)

X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

((443, 2736), (443, 58), (149, 2736), (149, 58), (149, 2736), (149, 58))

In [11]:
y_data_val.columns

Index(['Matt Set Label', 'Previous run_order', 'Previous MV', 'Study ID',
       'is Pediatric', 'Cohort Label v0', 'Job ID', 'Cancer Risk', 'Column',
       'Polarity', 'Sex', 'Subject ID', 'Age', 'Diagnosis', 'Region',
       'Smoking Status', 'Sample_Class', 'Ref', 'run_order', 'timestamp',
       'runtime_hour', 'batch_id', 'BMI', 'Race', 'Treatment',
       'Diagnosis Type', 'is Female', 'Type', 'Ethnicity', 'mzml path',
       'Batch', 'SARS-CoV-2 Positive', 'Matt Set', 'Set', 'NIVO OS', 'EVER OS',
       'Benefit BINARY', 'Nivo Benefit BINARY', 'MSKCC BINARY', 'IMDC BINARY',
       'Benefit ORDINAL', 'MSKCC ORDINAL', 'IMDC ORDINAL', 'OS', 'OS_Event',
       'Timepoint', 'Dose (mg/kg)', 'PFS', 'PFS_Event', 'ORR', 'Benefit',
       'Clinical Trial Phase', 'Diagnosis Details', 'Emphysema/COPD', 'IMDC',
       'MSKCC', 'Prior_2', 'Benzene_Exposure_Category'],
      dtype='object')

## Predicting binary tasks

#### IMDC Binary

In [12]:
import importlib
import finetune.latent_task_predict

importlib.reload(finetune.latent_task_predict)
from finetune.latent_task_predict import log_reg_multi_class


task = 'IMDC BINARY'

best_val_f1, best_val_auc, test_f1, test_auc= log_reg_multi_class(task, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)


best_val_f1, best_val_auc, test_f1, test_auc

Test Accuracy with best model: 0.8070
Test AUC with best model: 0.8990
Test F1 Score with best model: 0.8081


(0.8699778669767791,
 0.9044289044289046,
 0.8080880166517991,
 0.8989769820971867)

## Predicting categorical tasks 

In [13]:

results = []

# Loop over each task
for task in task_list_cat:
    print(f"Evaluating task: {task}")
    
    # Call the log_reg_multi_class function and get the results
    best_val_f1, best_val_auc, test_f1, test_auc= log_reg_multi_class(task, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)

    # Append the results to the list
    results.append({
        'Task': task,
        'Best Val F1': best_val_f1,
        'Best Val AUC': best_val_auc,
        'Test F1': test_f1,
        'Test AUC': test_auc
    })

# Convert the list of results to a pandas DataFrame
results_df = pd.DataFrame(results)


results_df

Evaluating task: Benefit BINARY
Test Accuracy with best model: 0.5682
Test AUC with best model: 0.5647
Test F1 Score with best model: 0.5684
Evaluating task: Nivo Benefit BINARY
Test Accuracy with best model: 0.5490
Test AUC with best model: 0.6815
Test F1 Score with best model: 0.5362
Evaluating task: MSKCC BINARY
Test Accuracy with best model: 0.8690
Test AUC with best model: 0.9178
Test F1 Score with best model: 0.8648
Evaluating task: IMDC BINARY
Test Accuracy with best model: 0.8070
Test AUC with best model: 0.8990
Test F1 Score with best model: 0.8081
Evaluating task: Benefit ORDINAL
Test Accuracy with best model: 0.4094
Test AUC with best model: 0.5607
Test F1 Score with best model: 0.4015
Evaluating task: MSKCC ORDINAL
Test Accuracy with best model: 0.6242
Test AUC with best model: 0.7658
Test F1 Score with best model: 0.6213
Evaluating task: IMDC ORDINAL
Test Accuracy with best model: 0.6573
Test AUC with best model: 0.6659
Test F1 Score with best model: 0.5829
Evaluating task

,Task,Best Val F1,Best Val AUC,Test F1,Test AUC
0,Benefit BINARY,0.635987,0.617054,0.568405,0.564700
1,Nivo Benefit BINARY,0.581743,0.552469,0.536221,0.681538
2,MSKCC BINARY,0.798306,0.859788,0.864770,0.917833
3,IMDC BINARY,0.869978,0.904429,0.808088,0.898977
4,Benefit ORDINAL,0.437934,0.574582,0.401520,0.560666
5,MSKCC ORDINAL,0.654688,0.794837,0.621294,0.765812
6,IMDC ORDINAL,0.606387,0.744794,0.582949,0.665920
7,ORR,0.471305,0.634932,0.361323,0.572683
8,Benefit,0.437934,0.574582,0.401520,0.560666
9,IMDC,0.579727,0.747130,0.546841,0.661997


## Predicting Survival task

### OS - L1 Cox

In [9]:
import finetune.latent_task_predict
importlib.reload(finetune.latent_task_predict)
from finetune.latent_task_predict import cox_proportional_hazards_l1_sksurv


task = 'OS'
task_event = 'OS_Event'


#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards_l1_sksurv(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: ConvergenceWarning: Optimization terminated early, you might want to increase the number of iterations (max_iter=100000).
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)


(0.6847947856197169, 0.6247225025227043, 0.1)

### Nivo OS - L1 COX

In [6]:
import finetune.latent_task_predict
importlib.reload(finetune.latent_task_predict)
from finetune.latent_task_predict import cox_proportional_hazards_l1_sksurv


task = 'NIVO OS'
task_event = 'OS_Event'


#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards_l1_sksurv(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: ConvergenceWarning: Optimization terminated early, you might want to increase the number of iterations (max_iter=100000).
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: ConvergenceWarning: Optimization terminated early, you might want to increase the number of iterations (max_iter=100000).
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)


(0.6293835363602805, 0.6049519586104952, 0.1)

###  Evero OS - L1 COX

In [7]:
import finetune.latent_task_predict
importlib.reload(finetune.latent_task_predict)
from finetune.latent_task_predict import cox_proportional_hazards_l1_sksurv


task = 'EVER OS'
task_event = 'OS_Event'


#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards_l1_sksurv(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: ConvergenceWarning: Optimization terminated early, you might want to increase the number of iterations (max_iter=100000).
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: ConvergenceWarning: Optimization terminated early, you might want to increase the number of iterations (max_iter=100000).
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)


(0.7063419117647058, 0.6736794986571173, 0.1)

### PFS - L1 COx 

In [8]:
import finetune.latent_task_predict
importlib.reload(finetune.latent_task_predict)
from finetune.latent_task_predict import cox_proportional_hazards_l1_sksurv

task = 'PFS'
task_event = 'PFS_Event'

#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards_l1_sksurv(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params

/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: ConvergenceWarning: Optimization terminated early, you might want to increase the number of iterations (max_iter=100000).
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)
/home/leilapirhaji/mz_embed_engine/ml/finetune/latent_task_predict.py:268: UserWarning: all coefficients are zero, consider decreasing alpha.
  model.fit(X_train_clean, Y_train)


(0.5668301001918568, 0.5432960893854749, 0.1)

# Fine-tune New splits

## importing the data

In [14]:

#input data
input_data_location='/home/leilapirhaji/PROCESSED_DATA_S_8.1.1'


#tasks to predict using encoder
task_list_cat=['Benefit BINARY', 'Nivo Benefit BINARY', 'MSKCC BINARY', 'IMDC BINARY', 'Benefit ORDINAL', 'MSKCC ORDINAL', 'IMDC ORDINAL', 'ORR', 'Benefit', 'IMDC', 'MSKCC', 'Prior_2' ]

#survival tasks
task_list_survival=[ 'OS', 'OS_Event', 'PFS', 'PFS_Event' ]


#get fine-tuning input data 
(X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)=get_finetune_input_data(input_data_location)

X_data_train.shape, y_data_train.shape, X_data_val.shape, y_data_val.shape, X_data_test.shape, y_data_test.shape

((443, 2736), (443, 58), (149, 2736), (149, 58), (149, 2736), (149, 58))

## predicticting categorical tasks

In [15]:
results = []

# Loop over each task
for task in task_list_cat:
    print(f"Evaluating task: {task}")
    
    # Call the log_reg_multi_class function and get the results
    best_val_f1, best_val_auc, test_f1, test_auc= log_reg_multi_class(task, X_data_train, y_data_train, X_data_val, y_data_val, X_data_test, y_data_test)

    # Append the results to the list
    results.append({
        'Task': task,
        'Best Val F1': best_val_f1,
        'Best Val AUC': best_val_auc,
        'Test F1': test_f1,
        'Test AUC': test_auc
    })

# Convert the list of results to a pandas DataFrame
results_df = pd.DataFrame(results)


results_df

Evaluating task: Benefit BINARY
Test Accuracy with best model: 0.6049
Test AUC with best model: 0.6280
Test F1 Score with best model: 0.6049
Evaluating task: Nivo Benefit BINARY
Test Accuracy with best model: 0.5625
Test AUC with best model: 0.6383
Test F1 Score with best model: 0.5558
Evaluating task: MSKCC BINARY
Test Accuracy with best model: 0.8395
Test AUC with best model: 0.8868
Test F1 Score with best model: 0.8349
Evaluating task: IMDC BINARY
Test Accuracy with best model: 0.8627
Test AUC with best model: 0.9373
Test F1 Score with best model: 0.8631
Evaluating task: Benefit ORDINAL
Test Accuracy with best model: 0.4027
Test AUC with best model: 0.5504
Test F1 Score with best model: 0.4045
Evaluating task: MSKCC ORDINAL
Test Accuracy with best model: 0.6040
Test AUC with best model: 0.7530
Test F1 Score with best model: 0.6000
Evaluating task: IMDC ORDINAL
Test Accuracy with best model: 0.6528
Test AUC with best model: 0.6554
Test F1 Score with best model: 0.6111
Evaluating task

,Task,Best Val F1,Best Val AUC,Test F1,Test AUC
0,Benefit BINARY,0.646289,0.669839,0.604938,0.628049
1,Nivo Benefit BINARY,0.536817,0.560000,0.555781,0.638261
2,MSKCC BINARY,0.870175,0.903704,0.834868,0.886831
3,IMDC BINARY,0.811321,0.868116,0.863066,0.937304
4,Benefit ORDINAL,0.429102,0.599409,0.404461,0.550445
5,MSKCC ORDINAL,0.622295,0.759465,0.600048,0.752987
6,IMDC ORDINAL,0.602091,0.694690,0.611054,0.655426
7,ORR,0.366766,0.605667,0.421550,0.561036
8,Benefit,0.429102,0.599409,0.404461,0.550445
9,IMDC,0.567700,0.708432,0.574011,0.646846


## Perdicting Survival Tasks

### OS -L1 Cox

In [ ]:
import finetune.latent_task_predict
importlib.reload(finetune.latent_task_predict)
from finetune.latent_task_predict import cox_proportional_hazards_l1_sksurv


task = 'OS'
task_event = 'OS_Event'


#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards_l1_sksurv(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params

### Nivo OS - L1 Cox

In [ ]:
import finetune.latent_task_predict
importlib.reload(finetune.latent_task_predict)
from finetune.latent_task_predict import cox_proportional_hazards_l1_sksurv


task = 'NIVO OS'
task_event = 'OS_Event'


#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards_l1_sksurv(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params

### Ever OS - L1 COx

In [ ]:
import finetune.latent_task_predict
importlib.reload(finetune.latent_task_predict)
from finetune.latent_task_predict import cox_proportional_hazards_l1_sksurv


task = 'EVER OS'
task_event = 'OS_Event'


#Pass the filtered data to the model
(best_val_c_index, best_test_c_index, best_params)= cox_proportional_hazards_l1_sksurv(X_data_train, y_data_train[task], y_data_train[task_event], X_data_val, y_data_val[task], y_data_val[task_event], X_data_test, y_data_test[task], y_data_test[task_event])

best_val_c_index, best_test_c_index, best_params